#  Classification : Nearest Neighbors and Naive Bayes 

## Classification using Nearest Neighbors

(a) Perform k-Nearest neighbours on the given dataset($X_{knn}$ and $y_{knn}$: where $X_{knn}$ stores feature vectors representing the movies and  $y_{knn}$ stores the 0-1 labelling for each movie) for binary classification of movies, for classifiying whether a given movie is a comedy(label 1) or not a comedy(label 0) . Split the dataset into train(80%), validation(10%) and test sets(10%).Run k-Nearest neighbours for different k values (1,3,7,15,31,63). Select the k, using validation set, which returns the best accuracy score. 

(i)  Report all the validation accuracies for all the values of k. 
<br>(ii) Report accuracy score by performing k-NN on the test dataset using the best chosen k value. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import tqdm

X = pd.read_csv('X_knn.csv', sep=' ', header=None)
y = pd.read_csv('y_knn.csv', header=None)
X_train, Xdash, y_train, ydash = train_test_split(X, y, train_size=0.8, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(Xdash, ydash, train_size=0.5, random_state=0)

k = [1,3,7,15,31,63]
acc = []
for i in tqdm(range(len(k))):
    clf = KNeighborsClassifier(n_neighbors=k[i])
    clf.fit(X_train, np.array(y_train).reshape(-1))
    acc.append(clf.score(X_val, y_val))
    
for i in range(len(k)):
    print('Value of k: %f, Accuracy on Validation set: %f' %(k[i], acc[i]))

Value of k: 1.000000, Accuracy on Validation set: 0.822000
Value of k: 3.000000, Accuracy on Validation set: 0.845000
Value of k: 7.000000, Accuracy on Validation set: 0.849000
Value of k: 15.000000, Accuracy on Validation set: 0.841000
Value of k: 31.000000, Accuracy on Validation set: 0.842000
Value of k: 63.000000, Accuracy on Validation set: 0.833000


In [2]:

clf = KNeighborsClassifier(n_neighbors=7)
clf.fit(X_train, np.array(y_train).reshape(-1))
acc = clf.score(X_test, y_test)
print('Accuracy on test set for k = 7: ', acc)

Accuracy on test set for k = 7:  0.848


(b) State why using an even value of k in k-NN should not be chosen

To prevent equal number of predictions for the two classes, we prefer odd value of k

## Learning Naive Bayes' classifier  

### From Continuous Distribution of data

Here, the distribution of the data( $X$ represents the datapoints and $Y$ represents the 0-1 binary-class label; where 0 being the negative class and 1 being the positive class) is already known.
<br>Consider the following one-dimensional(1-D) Gaussian distributions where means and variances are unknown. You need to estimate means($\mu_-$: for negative class and  $\mu_+$: for positive class) and variances ($\sigma^{2}_{-}$: for negative class and $\sigma^{2}_+$: for positive class) from the given data : 
<br> (1) Assume $X|Y_{Y=0} \sim \mathcal{N}(\mu_- , \sigma^{2}_-)$ 
<br>(2) Assume $X|Y_{Y=1} \sim \mathcal{N}(\mu_+ , \sigma^{2}_+)$


*Generating artificial datasets in the next cell *

In [3]:
## This cell is for generating datasets. Students should not change anything in this cell. 
## You can compare your mean and variance estimates by the actual ones used to generate these datasets

import numpy as np
X_pos = np.random.randn(1000,1)+np.array([[2.]])
X_neg = np.random.randn(1000,1)+np.array([[4.]])
X_train_pos = X_pos[:900]
X_train_neg = X_neg[:900]
X_test_pos = X_pos[900:]
X_test_neg = X_neg[900:]
X_train = np.concatenate((X_train_pos, X_train_neg), axis=0)
X_test = np.concatenate((X_test_pos, X_test_neg), axis=0)
Y_train = np.concatenate(( np.ones(900),np.zeros(900) ))
Y_test = np.concatenate(( np.ones(100), np.zeros(100) ))

## X_train, X_test, Y_train, Y_test are your datasets to work with ####



<br>**Instructions to follow for learning a Baeysian classifier:** *(Code the formulae for estimating the different parameters yourself)*
<br> a)Utilize the training dataset to estimate the means($\hat{\mu_+}$,$\hat{\mu_-}$) and variances($\hat{\sigma^{2}_+}$, $\hat{\sigma^{2}_-}$) for both positive and negative classes  
b)Estimate the prior probability: $P(Y=1)$  ⟶ which could be referred to as: $\hat{a}$ 
<br>c)Estimate the classifier funtion/posterior probability:  $P(Y=1|X = x)$  ⟶ which could be referred to as $\hat{\eta(x)}$
<br>d)Find out the threshold value($x^*$) for classification by equating the estimated classifier function($\hat{\eta(x)}$)  with threshold probability of 0.5
<br>e)Classify the test dataset into the two classes using this threshold value($x^*$) and find out the **accuracy** of the prediction 

Return back:  $\hat{\mu_+}$, $\hat{\mu_-}$, $\hat{\sigma^{2}_+}$, $\hat{\sigma^{2}_-}$, $\hat{a}$, $x^*$ and accuracy from the code written 

*Hint: $X|Y_{Y=0} \sim \mathcal{N}(\mu_- , \sigma^{2}_-)$ implies $P_{X|Y=0} = \mathcal{N}(\mu_- , \sigma^{2}_-) $*


In [4]:
import scipy.stats
from sklearn.metrics import accuracy_score

mu_pos=X_train_pos.mean()
mu_neg=X_train_neg.mean()
mu_train=X_train.mean()

sig_pos = (np.array([(x - mu_pos) ** 2 for x in X_train_pos]).sum()/(len(X_train_pos)-1))**0.5
sig_neg = (np.array([(x - mu_neg) ** 2 for x in X_train_neg]).sum()/(len(X_train_neg)-1))**0.5
sig_train = (np.array([(x - mu_train) ** 2 for x in X_train]).sum()/(len(X_train)-1))**0.5

a = X_train[:900].shape[0]/(X_train.shape[0])    


In [5]:

def post_prob(x):
    return ((scipy.stats.norm(mu_pos, sig_pos).pdf(x)*a)/scipy.stats.norm(mu_train, sig_train).pdf(x))

min_difference=100000
x_threshold=100000
for i in X_train:
    difference=abs(post_prob(i)-.5)
    if(difference <min_difference):
        min_difference=difference
        x_threshold=i

Y_test_predicted=[]
for i in range(len(X_test)):
    if(X_test[i][0] < x_threshold):
        Y_test_predicted.append(1)
    else:
        Y_test_predicted.append(0)

In [6]:
print("Estimated mean for positive class",mu_pos)       
print("Estimated mean for negative class",mu_neg)    
print("Estimated Standard Deviation for positive class",sig_pos)   
print("Estimated Standard Deviation for negative class",sig_neg)   
print("Threshold value for x: ",x_threshold)   
print("Accuracy :", accuracy_score(Y_test,Y_test_predicted))

Estimated mean for positive class 1.9999833359784351
Estimated mean for negative class 3.9677242071345944
Estimated Standard Deviation for positive class 1.0521529162205658
Estimated Standard Deviation for negative class 0.9881698804624363
Threshold value for x:  [2.81996095]
Accuracy : 0.84


### From Discrete distribution of data

Unlike the first exercise for learning the Naive Bayes' classifier where we dealt with continuous distribution of data, here you need to work with discrete data, which means finding Probability Mass Distribution(PMF). 

Age  | Income | Status  | Buy
-----|--------|-------- |----
<=20 |  low   | students| yes
<=20 |  high  | students| yes
<=20 | medium | students| no
<=20 | medium | married | no
<=20 |  high  | married | yes
21-30|  low   | married | yes
21-30|  low   | married | no 
21-30| medium | students| no
21-30|  high  | students| yes
 >30 |  high  | married | no
 >30 |  high  | married | yes
 >30 | medium | married | yes
 >30 | medium | married | no
 >30 | medium | students| no
 
Consider the train dataset above. Take any random datapoint ($X_{i}$) where $X_{i} = (X_{i,1} = Age,X_{i,2} = Income,X_{i,3} = Status)$ and its corresponding label 

($Y_{i} = Buy$). A "yes" in Buy corresponds to label-1 and a "no" in Buy corresponds to label-0.

<br>**Instructions to follow for learning a Baeysian classifier:** *(Code the formulae for estimating the different parameters yourself)*
<br> a)Estimate the prior probability: $P(Y=1)$  ⟶ which could be referred to as: $\hat{a}$   
b)Estimate the likelihood for each feature:  $P(X_{i,j} = x |Y = y_{i})$, where $ i$=datapoint counter, $j \in \{1,2,3\}$ and $y_{i} \in \{0,1\}$ 
<br>c)Estimate the total likelihood: $P(X_{i} = x |Y = y_{i})$  
d)Calculate the posterior probability: $P(Y = 1|X_{i} = x_{test} )$ = $p_{test}$ where $x_{test} = (Age = 21-30, Income= medium, Status = married)$


Return back: $\hat{a}$, total likelihood and $p_{test}$ 


In [7]:

data = {
    "Age": ["<=20","<=20","<=20","<=20","<=20","21-30","21-30","21-30","21-30",">30",">30",">30",">30",">30"],
    "Income": [ "low", "high", "medium", "medium", "high",  "low",  "low", "medium", "high", "high", "high", "medium", "medium", "medium"],
    "Status": ["students", "students", "students", "married", "married", "married", "married", "students", "students", "married", "married", "married", "married", "students"],
    "Buy": ["yes", "yes", "no", "no", "yes", "yes", "no", "no", "yes", "no", "yes", "yes", "no", "no"]
}
df = pd.DataFrame(data)

prior = (df['Buy'] == 'yes').sum()/len(df)

pos_df = df[df['Buy'] == 'yes']
age_pos = dict(pos_df['Age'].value_counts()/len(pos_df))
income_pos = dict(pos_df['Income'].value_counts()/len(pos_df))
status_pos = dict(pos_df['Status'].value_counts()/len(pos_df))

likelihoods = []

for i in range(len(df)):
    age = df.at[i, 'Age']
    income = df.at[i, 'Income']
    status = df.at[i, 'Status']
    likelihoods.append(age_pos[age] * income_pos[income] * status_pos[status])

p_test = age_pos['21-30'] * income_pos['medium'] * status_pos['married'] * prior
df['Likelihood'] = likelihoods

In [8]:
print("Probability that x_test will buy:",p_test)
print("Prior probability of buying: ",prior)
print(df)

Probability that x_test will buy: 0.011661807580174925
Prior probability of buying:  0.5
      Age  Income    Status  Buy  Likelihood
0    <=20     low  students  yes    0.052478
1    <=20    high  students  yes    0.104956
2    <=20  medium  students   no    0.026239
3    <=20  medium   married   no    0.034985
4    <=20    high   married  yes    0.139942
5   21-30     low   married  yes    0.046647
6   21-30     low   married   no    0.046647
7   21-30  medium  students   no    0.017493
8   21-30    high  students  yes    0.069971
9     >30    high   married   no    0.093294
10    >30    high   married  yes    0.093294
11    >30  medium   married  yes    0.023324
12    >30  medium   married   no    0.023324
13    >30  medium  students   no    0.017493
